In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow import keras
from tensorflow.keras import layers as Layer
from tensorflow.keras.metrics import Recall, Precision, BinaryAccuracy, CategoricalAccuracy, SparseCategoricalAccuracy
from tensorflow.keras.regularizers import L1L2
from timeit import default_timer as timer

In [ ]:
x2 = df_clear3
y2 = df_target2["Type"]

# 스케일링
scaler2 = StandardScaler()
x_scaler2 = scaler.fit_transform(x2)
x_scaler2 = pd.DataFrame(x_scaler2, columns=x2.columns)

# 훈련
x_train2, x_test2, y_train2, y_test2 = train_test_split(x_scaler2, y2, test_size=0.2)
encoder = LabelEncoder()
y_train2, y_test2 = encoder.fit_transform(y_train2), encoder.fit_transform(y_test2)
x_train2.shape, x_test2.shape, y_train2.shape, y_test2.shape

((1013, 20), (254, 20), (1013,), (254,))

In [ ]:
encoder.classes_

array(['Bumps', 'Dirtiness', 'K_Scatch', 'Pastry', 'Stains', 'Z_Scratch'],
      dtype=object)

In [ ]:
callback = [
    tf.keras.callbacks.EarlyStopping(monitor="loss",min_delta = 0.001, patience=100),
    tf.keras.callbacks.ModelCheckpoint(filepath=check_path, save_weights_only=True, monitor='loss', mode='min', save_best_only=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2, patience=100, min_lr=0.01),
    TimingCallback()
]

In [ ]:
class TimingCallback(keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        print("Starting training")
        self.starttime = timer()

    def on_train_end(self, logs=None):
        print("End of training, took {} seconds".format(timer()-self.starttime))

    def on_epoch_end(self, epoch, logs=None):
        if epoch % 10 == 0:
          print("Epoch is {} and {} seconds passed".format(epoch, timer()-self.starttime))


def multi_model(drop_rate = 0.1):
  model = Sequential()
  model.add(Layer.Dense(12, activation = 'relu'))
  model.add(Layer.Dropout(0.1))

  model.add(Layer.Dense(4, activation = 'relu'))
  model.add(Layer.Dropout(0.1))


  model.add(Layer.Dense(6, activation = 'softmax'))

  metrics = [
      SparseCategoricalAccuracy(name = 'accuracy') # Accuracy를 사용 안 하는 이유는 Accuracy가 이상하세 나왔기 때문.
  ]

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
                 loss="sparse_categorical_crossentropy",
                 metrics=metrics)

  return model

model3 = multi_model()
history3 = model3.fit(x=x_train2, y=y_train2, batch_size=1024,
                      epochs=1000, shuffle=True,
                      validation_split=0.1,callbacks=callback, verbose = 0)

Starting training
Epoch is 0 and 1.4110541169975477 seconds passed
Epoch is 10 and 1.7973434039995482 seconds passed
Epoch is 20 and 2.225679486997251 seconds passed
Epoch is 30 and 2.5748542699984682 seconds passed
Epoch is 40 and 2.976891676997184 seconds passed
Epoch is 50 and 3.377469976996508 seconds passed
Epoch is 60 and 3.7809661949977453 seconds passed
Epoch is 70 and 4.134420184996998 seconds passed
Epoch is 80 and 4.526497031998588 seconds passed
Epoch is 90 and 4.938458445998549 seconds passed
Epoch is 100 and 5.317124580997188 seconds passed
Epoch is 110 and 5.7524217369973485 seconds passed
Epoch is 120 and 6.277178413998627 seconds passed
Epoch is 130 and 6.834885083997506 seconds passed
Epoch is 140 and 7.352710395996837 seconds passed
Epoch is 150 and 7.928574410998408 seconds passed
Epoch is 160 and 8.462920368998311 seconds passed
Epoch is 170 and 8.973048344996641 seconds passed
Epoch is 180 and 9.51871416999711 seconds passed
Epoch is 190 and 10.101486711999314 sec

In [ ]:
y_test_pred = model3.predict(x_test2)

y_test_pred = np.argmax(y_test_pred, axis=-1)

y_test_pred

8/8 [==============================] - 0s 2ms/step


array([0, 2, 0, 0, 2, 3, 2, 2, 0, 3, 0, 2, 3, 2, 2, 0, 3, 0, 0, 5, 5, 3,
       0, 0, 4, 0, 0, 3, 1, 2, 5, 0, 0, 5, 3, 2, 4, 0, 0, 4, 2, 0, 2, 0,
       0, 2, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 2, 0, 3, 3, 0, 0, 1, 0, 0, 2,
       0, 0, 2, 0, 2, 2, 3, 2, 3, 1, 0, 0, 1, 2, 2, 3, 0, 2, 3, 1, 3, 2,
       0, 0, 0, 2, 2, 2, 5, 4, 5, 2, 0, 5, 3, 0, 1, 4, 4, 2, 4, 5, 0, 2,
       2, 2, 2, 0, 0, 3, 3, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 2, 3, 0, 0, 0,
       5, 0, 5, 5, 2, 0, 2, 2, 5, 2, 5, 5, 0, 2, 0, 5, 2, 0, 2, 1, 5, 5,
       0, 0, 3, 2, 0, 2, 2, 2, 0, 0, 2, 5, 2, 2, 5, 2, 5, 2, 2, 2, 5, 2,
       3, 3, 0, 1, 3, 0, 5, 0, 2, 0, 0, 4, 0, 5, 3, 0, 0, 5, 3, 0, 2, 2,
       5, 0, 0, 5, 0, 3, 5, 4, 0, 5, 3, 4, 2, 0, 2, 2, 3, 0, 5, 2, 2, 2,
       3, 5, 5, 2, 0, 2, 0, 5, 2, 5, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       4, 0, 2, 0, 4, 2, 0, 3, 3, 2, 2, 0])

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test2, y_test_pred)

cm = pd.DataFrame(cm)
cm.columns = ['Bumps', 'Dirtiness', 'K_Scatch', 'Pastry', 'Stains', 'Z_Scratch']

cm

,Bumps,Dirtiness,K_Scatch,Pastry,Stains,Z_Scratch
0,80,0,1,2,2,3
1,5,8,0,1,0,0
2,1,0,70,0,1,1
3,8,0,0,27,0,1
4,0,0,1,0,10,0
5,3,0,1,0,0,28


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test2, y_test_pred))

              precision    recall  f1-score   support

           0       0.82      0.91      0.86        88
           1       1.00      0.57      0.73        14
           2       0.96      0.96      0.96        73
           3       0.90      0.75      0.82        36
           4       0.77      0.91      0.83        11
           5       0.85      0.88      0.86        32

    accuracy                           0.88       254
   macro avg       0.88      0.83      0.84       254
weighted avg       0.88      0.88      0.88       254

